<a href="https://colab.research.google.com/github/olonok69/Deep_learning_Image_Caption/blob/master/spacy_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy==2.3.2 
!python -m spacy download en_core_web_lg
!pip install nltk

     |████████████████████████████████| 10.0MB 2.7MB/s 
     |████████████████████████████████| 2.1MB 53.4MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 782.7MB 1.1MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.3.1-cp36-none-any.whl size=782936125 sha256=bdb682366e7a72444e9bc1b18c14b46876171841a71ee01c7e8d01ab2907bd19
  Stored in directory: /tmp/pip-ephem-wheel-cache-5op2ne2w/wheels/ce/4d/1b/bc6cabb6df139c5f0318927be3ae9e51363fb44d6ea328d3f4
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import spacy
import en_core_web_lg
nlp = en_core_web_lg.load()
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
#spacy_stopwords.remove('a')
import re
import datetime

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
datap= pd.read_csv("/content/drive/My Drive/colab/datasets/defects_2.csv")

In [ ]:
datap.columns

Index(['ID', 'Survey_Setup_Id', 'Survey_Setup_Name', 'Survey_Block',
       'Location', 'Location_Operational_Status', 'Element_Name',
       'Feature_Name', 'Action_Comment', 'Response_Priority', 'Impact_Score',
       'Prob_Score', 'Risk_Score', 'Condition_Quantity', 'Unit_Of_Measure',
       'url_x', 'Multiple_Photos', 'Action_Response', 'Comment',
       'picture_file', 'url_y', 'status_download', 'alternate', 'keywords',
       'correct_Action'],
      dtype='object')

In [ ]:
datap[['keywords','correct_Action']].head(20)

,keywords,correct_Action
0,"['movement', 'rear', 'shleter']",There appears to be signs of ground erosion/mo...
1,"['exam', 'ext', 'integrity']",CCBG. Consider exposing sample areas of buried...
2,"['copers', 'approx', '%']","Several copers loose, fractured or missing fro..."
3,"['platform', 'plywood', 'poin']",Surface breaking up and plywood delaminating a...
4,"['surface', 'signs', 'cracks']",Repaired surface showing signs of cracks/break...
5,"['line', 'platform', 'edge']",The paint forming the white line is worn and f...
6,"['fence', 'vegetation', 'degrees']",Fence overgrown with vegetation to various deg...
7,"['timber', 'ground', 'level']",Bases to timber trestle posts are either burie...
8,"['inspection', 'tray', 'cables']",0.00-8.95m from ramp: 300mm Cable tray fixed t...
9,"['riser', 'spalling', 'erosion']",57.4-146.30m from ramp: Erosion & spalling to ...


In [ ]:
def remove_jpg(text):
  
  text = re.sub(r'jpg','',text)
  return text
def remove_punkt(row):
  text=row.correct_Action_clean
  text = re.sub(r'[^a-zA-Z\s]',' ',text)
  return text
def spacy_action_description(texto, matcher, nlp):
  #texto=row.correct_Action_clean
  doc = nlp(texto)
  # call the matcher to find matches 
  matches = matcher(doc)
  spans = [doc[start:end] for _, start, end in matches]
  out =filter_spans(spans)
  out2=[s.text for s in out ]
  return out2
  

In [ ]:

from spacy.matcher import Matcher
from spacy.util import filter_spans

pattern = [{'POS': 'VERB', 'OP': '?'},
           {'POS': 'ADV', 'OP': '*'},
           {'POS': 'AUX', 'OP': '*'},
           {'POS': 'VERB', 'OP': '+'}]

# instantiate a Matcher instance
matcher = Matcher(nlp.vocab)
matcher.add("Verb phrase", None, pattern)

In [ ]:
datap['spacy_verb']=''

In [ ]:
# correct mispelled words on text
# Iterate on the index as is 20 times faster than apply the fucntion
tim1= datetime.datetime.now()
for i in datap.index:
 
    empty_list=[]
    texto= datap.at[i,'correct_Action']
    try:
      texto_clean=remove_jpg(texto)
    except:
      texto_clean=texto
    datap.at[i,'correct_Action']=texto_clean 
    try:
      datap.at[i,'spacy_verb']=spacy_action_description(texto,matcher,nlp)
    except:
       datap.at[i,'spacy_verb']=empty_list
    if i%5000==0:
      print (i)


tim2= datetime.datetime.now()        
print(tim2-tim1)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000


In [ ]:
datap.to_csv("/content/drive/My Drive/colab/datasets/defects_2.csv", index=False)